In [1]:
import tensorflow as tf
import networkx as nx
import numpy as np

import data.input_samples as inpu
import models.graph_nn as grap
import util.plotting as plut
import data.input_samples as inpu

%load_ext autoreload

%autoreload 2

In [2]:
X, A_tilde, A, train_mask, valid_mask = inpu.make_karate_data_autoencoder()
nodes_n = len(X)

In [3]:
train_mask

array([[ True,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False,  True],
       [False, False,  True, ...,  True,  True,  True],
       ...,
       [ True, False,  True, ...,  True,  True,  True],
       [ True, False, False, ..., False,  True, False],
       [ True,  True, False, ..., False,  True,  True]])

In [10]:
gnn = grap.GraphAutoencoderKarate(nodes_n=nodes_n, feat_sz=nodes_n, activation=tf.nn.tanh)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_features (InputLa [(None, 34, 34)]     0                                            
__________________________________________________________________________________________________
encoder_input_adjacency (InputL [(None, 34, 34)]     0                                            
__________________________________________________________________________________________________
graph_convolution_3 (GraphConvo (None, 34, 4)        136         encoder_input_features[0][0]     
                                                                 encoder_input_adjacency[0][0]    
__________________________________________________________________________________________________
graph_convolution_4 (GraphConvo (None, 34, 4)        16          graph_convolution_3[0]

In [11]:
z, A_pred = gnn((X, A_tilde))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [12]:
# predicted adjacencies from untrained model
tf.nn.sigmoid(A_pred)

<tf.Tensor: shape=(34, 34), dtype=float32, numpy=
array([[0.51190925, 0.504779  , 0.5049792 , ..., 0.49845704, 0.502992  ,
        0.50387293],
       [0.504779  , 0.50194365, 0.5020104 , ..., 0.49935934, 0.5012125 ,
        0.50158477],
       [0.5049792 , 0.5020104 , 0.50208753, ..., 0.49934462, 0.5012565 ,
        0.5016339 ],
       ...,
       [0.49845704, 0.49935934, 0.49934462, ..., 0.5002177 , 0.49960253,
        0.49947286],
       [0.502992  , 0.5012125 , 0.5012565 , ..., 0.49960253, 0.5007571 ,
        0.500987  ],
       [0.50387293, 0.50158477, 0.5016339 , ..., 0.49947286, 0.500987  ,
        0.5012956 ]], dtype=float32)>

In [13]:
gnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), run_eagerly=True)

In [14]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)]
gnn.fit((X[np.newaxis,:,:], A_tilde[np.newaxis,:,:], train_mask[np.newaxis,:,:]), A[np.newaxis,:,:], epochs=100, validation_data=((X[np.newaxis,:,:], A_tilde[np.newaxis,:,:], valid_mask[np.newaxis,:,:]), A[np.newaxis,:,:]), callbacks=callbacks)

Epoch 1/100
1/1 [==============================] - 0s 65ms/step - val_loss: 0.6929
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - val_loss: 0.6925
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - val_loss: 0.6922
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - val_loss: 0.6918
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - val_loss: 0.6914
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - val_loss: 0.6908
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - val_loss: 0.6900
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - val_loss: 0.6890
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - val_loss: 0.6876
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - val_loss: 0.6860
Epoch 11/100
1/1 [==============================] - 0s 22ms/step - val_loss: 0.6839
Epoch 12/100
1/1 [==============================] - 0s 29ms/step - val_loss: 0.6815
E

Epoch 82/100
1/1 [==============================] - 0s 26ms/step - val_loss: 0.6452
Epoch 83/100
1/1 [==============================] - 0s 25ms/step - val_loss: 0.6452
Epoch 84/100
1/1 [==============================] - ETA: 0s
Epoch 00084: ReduceLROnPlateau reducing learning rate to 2.441406195430318e-06.
1/1 [==============================] - 0s 22ms/step - val_loss: 0.6452
Epoch 85/100
1/1 [==============================] - 0s 25ms/step - val_loss: 0.6452
Epoch 86/100
1/1 [==============================] - 0s 24ms/step - val_loss: 0.6452
Epoch 87/100
1/1 [==============================] - 0s 26ms/step - val_loss: 0.6452
Epoch 88/100
1/1 [==============================] - 0s 25ms/step - val_loss: 0.6452
Epoch 89/100
1/1 [==============================] - ETA: 0s
Epoch 00089: ReduceLROnPlateau reducing learning rate to 1.220703097715159e-06.
1/1 [==============================] - 0s 22ms/step - val_loss: 0.6452
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - val_los

In [15]:
z, A_pred = gnn((X, A_tilde))

In [16]:
tf.nn.sigmoid(A_pred)

<tf.Tensor: shape=(34, 34), dtype=float32, numpy=
array([[0.8438018 , 0.7924925 , 0.63885754, ..., 0.28950018, 0.18531656,
        0.18551764],
       [0.7924925 , 0.7437489 , 0.6116044 , ..., 0.32863334, 0.23555595,
        0.23573309],
       [0.63885754, 0.6116044 , 0.54827976, ..., 0.424425  , 0.3771823 ,
        0.3772723 ],
       ...,
       [0.28950018, 0.32863334, 0.424425  , ..., 0.6175449 , 0.68764263,
        0.6875082 ],
       [0.18531656, 0.23555595, 0.3771823 , ..., 0.68764263, 0.78593796,
        0.7857522 ],
       [0.18551764, 0.23573309, 0.3772723 , ..., 0.6875082 , 0.7857522 ,
        0.7855673 ]], dtype=float32)>

In [20]:
A_reco = (tf.nn.sigmoid(A_pred) > 0.5).numpy().astype(int)

In [21]:
A_reco

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])